In [3]:
import sympy as sp

# Set the number of segments (N must be at least 4)
N = 2

# Create lists of unknown coefficients for each segment.
# For segment i (i=0,...,N-1) we have cubic polynomial coefficients: a_i, b_i, c_i, d_i.
a = [sp.symbols(f'a{i}', real=True) for i in range(N)]
b = [sp.symbols(f'b{i}', real=True) for i in range(N)]
c = [sp.symbols(f'c{i}', real=True) for i in range(N)]
d = [sp.symbols(f'd{i}', real=True) for i in range(N)]

# Build a list "unknowns" in the order: a0, b0, c0, d0, a1, b1, c1, d1, ... etc.
unknowns = []
for i in range(N):
    unknowns.extend([a[i], b[i], c[i], d[i]])

# Define symbols for the boundary conditions.
P0, V0, A0 = sp.symbols('P0 V0 A0', real=True)   # initial state: position, velocity, acceleration
Pf, Vf, Af = sp.symbols('Pf Vf Af', real=True)     # final state

# Define a symbol for the total time.
T_total = sp.symbols('T_total', positive=True, real=True)
# Add T_total to the list of unknowns since we want to solve for it as well.
unknowns.append(T_total)

# For each segment, the time duration is T_total divided by the number of segments, N.
T = [T_total / N for _ in range(N)]

# Build the list of equations.
eqs = []

# --- Initial conditions for segment 0 at t = 0 ---
eqs.append(sp.Eq(d[0], P0))         # p0(0)= d0 = P0
eqs.append(sp.Eq(c[0], V0))         # p0'(0)= c0 = V0
eqs.append(sp.Eq(2*b[0], A0))       # p0''(0)= 2*b0 = A0   => b0 = A0/2

# --- Continuity conditions for segments 0 to N-2 ---
for i in range(N - 1):
    # Position continuity: p_i(T_i)= a_i*T_i^3 + b_i*T_i^2 + c_i*T_i + d_i equals the start of segment i+1, i.e. d_{i+1}.
    eqs.append(sp.Eq(a[i]*T[i]**3 + b[i]*T[i]**2 + c[i]*T[i] + d[i], d[i+1]))
    # Velocity continuity: p_i'(T_i)= 3*a_i*T_i^2 + 2*b_i*T_i + c_i equals c_{i+1}.
    eqs.append(sp.Eq(3*a[i]*T[i]**2 + 2*b[i]*T[i] + c[i], c[i+1]))
    # Acceleration continuity: p_i''(T_i)= 6*a_i*T[i] + 2*b_i equals 2*b[i+1].
    eqs.append(sp.Eq(6*a[i]*T[i] + 2*b[i], 2*b[i+1]))

# --- Final conditions for segment N-1 at t = T_{N-1} ---
eqs.append(sp.Eq(a[N-1]*T[N-1]**3 + b[N-1]*T[N-1]**2 + c[N-1]*T[N-1] + d[N-1], Pf))
eqs.append(sp.Eq(3*a[N-1]*T[N-1]**2 + 2*b[N-1]*T[N-1] + c[N-1], Vf))
eqs.append(sp.Eq(6*a[N-1]*T[N-1] + 2*b[N-1], Af))

print("Number of unknowns:", len(unknowns))
print("Number of equations:", len(eqs))  # Should be 3N+3

# Use sp.solve for nonlinear systems:
solution = sp.solve(eqs, unknowns, dict=True)
# solution = sp.nonlinsolve(eqs, unknowns)
if not solution:
    print("No solution found!")
else:
    sol_tuple = solution[0]
    print("\n// C++ Code for the coefficients and T_total:")
    for var in unknowns:
        print("double {} = {};".format(var, sp.ccode(sol_tuple[var])))

# =======================
# 2. Compute Control Points
# =======================
#
# For each segment i (with duration T[i]) the cubic polynomial is:
#   p_i(t)= a_i*t^3 + b_i*t^2 + c_i*t + d_i, for t in [0, T[i]].
#
# We define the position control points as:
#   CP0 = p_i(0) = d_i,
#   CP1 = (c_i*T[i] + 3*d_i) / 3,
#   CP2 = (b_i*T[i]**2 + 2*c_i*T[i] + 3*d_i) / 3,
#   CP3 = p_i(T[i]) = a_i*T[i]**3 + b_i*T[i]**2 + c_i*T[i] + d_i.
#
# And for the derivative curves (defined on the normalized parameter u=t/T[i]):
#   Velocity: v_i(t)= 3*a_i*t^2+ 2*b_i*t+c_i.
#     -> Define v0 = c_i, v_mid = 3*a_i*(T[i]/2)**2 + 2*b_i*(T[i]/2) + c_i, vT = 3*a_i*T[i]**2+ 2*b_i*T[i] + c_i.
#   Acceleration: a_i(t)= 6*a_i*t+2*b_i.
#     -> a0 = 2*b_i, aT = 6*a_i*T[i] + 2*b_i.
#   Jerk: j_i(t)= 6*a_i (constant).
#
# Define helper functions:

def position_control_points(a_coef, b_coef, c_coef, d_coef, T_val):
    CP0 = d_coef
    CP1 = (c_coef*T_val + 3*d_coef) / 3
    CP2 = (b_coef*T_val**2 + 2*c_coef*T_val + 3*d_coef) / 3
    CP3 = sp.simplify(a_coef*T_val**3 + b_coef*T_val**2 + c_coef*T_val + d_coef)
    return (CP0, CP1, CP2, CP3)

def velocity_control_points(a_coef, b_coef, c_coef, d_coef, T_val):
    # Velocity: v(t)= 3*a*t^2+2*b*t+c.
    v0 = sp.simplify(c_coef)  # at t=0
    v_mid = sp.simplify(3*a_coef*(T_val/2)**2 + 2*b_coef*(T_val/2) + c_coef)
    vT = sp.simplify(3*a_coef*T_val**2 + 2*b_coef*T_val + c_coef)
    return (v0, v_mid, vT)

def acceleration_control_points(a_coef, b_coef, c_coef, d_coef, T_val):
    # Acceleration: a(t)= 6*a*t+2*b.
    a0_val = sp.simplify(2*b_coef)  # at t=0
    aT_val = sp.simplify(6*a_coef*T_val + 2*b_coef)
    return (a0_val, aT_val)

def jerk_control_point(a_coef, b_coef, c_coef, d_coef, T_val):
    # Jerk: j(t)= 6*a, constant.
    return sp.simplify(6*a_coef)

# Substitute the solution into the unknowns:
subs_dict = dict(zip(unknowns, sol_tuple))
# For convenience, define lists for each segment’s coefficients:
a_sol = [subs_dict[a[i]] for i in range(N)]
b_sol = [subs_dict[b[i]] for i in range(N)]
c_sol = [subs_dict[c[i]] for i in range(N)]
d_sol = [subs_dict[d[i]] for i in range(N)]

# Compute control points for each segment.
CP = []      # position control points for each segment
VelCP = []   # velocity control points (as a triple) for each segment
AccelCP = [] # acceleration control points (as a pair) for each segment
JerkCP = []  # jerk control point for each segment

for i in range(N):
    CP.append(position_control_points(a_sol[i], b_sol[i], c_sol[i], d_sol[i], T[i]))
    VelCP.append(velocity_control_points(a_sol[i], b_sol[i], c_sol[i], d_sol[i], T[i]))
    AccelCP.append(acceleration_control_points(a_sol[i], b_sol[i], c_sol[i], d_sol[i], T[i]))
    JerkCP.append(jerk_control_point(a_sol[i], b_sol[i], c_sol[i], d_sol[i], T[i]))

# =======================
# 3. Print the results in C++ code format
# =======================
print("\n// POSITION CONTROL POINTS:")
for i in range(N):
    print(f"// Segment {i}:")
    CPi = CP[i]
    for j, cp in enumerate(CPi):
        print("double CP{0}_{1} = {2};".format(j, i, sp.ccode(cp)))

print("\n// VELOCITY CONTROL POINTS:")
for i in range(N):
    print(f"// Segment {i}:")
    VCPi = VelCP[i]
    for j, cp in enumerate(VCPi):
        print("double vCP{0}_{1} = {2};".format(j, i, sp.ccode(cp)))

print("\n// ACCELERATION CONTROL POINTS:")
for i in range(N):
    print(f"// Segment {i}:")
    ACPi = AccelCP[i]
    for j, cp in enumerate(ACPi):
        print("double aCP{0}_{1} = {2};".format(j, i, sp.ccode(cp)))

print("\n// JERK CONTROL POINTS:")
for i in range(N):
    print(f"// Segment {i}:")
    print("double jCP0_{0} = {1};".format(i, sp.ccode(JerkCP[i])))


Number of unknowns: 9
Number of equations: 9

// C++ Code for the coefficients and T_total:
double a0 = ((13.0/72.0)*A0*P0*V0 + (5.0/72.0)*A0*P0*Vf - 1.0/24.0*A0*P0*sqrt(-12*A0*P0 + 12*A0*Pf + 12*Af*P0 - 12*Af*Pf + 9*pow(V0, 2) + 18*V0*Vf + 9*pow(Vf, 2)) - 13.0/72.0*A0*Pf*V0 - 5.0/72.0*A0*Pf*Vf + (1.0/24.0)*A0*Pf*sqrt(-12*A0*P0 + 12*A0*Pf + 12*Af*P0 - 12*Af*Pf + 9*pow(V0, 2) + 18*V0*Vf + 9*pow(Vf, 2)) - 1.0/72.0*Af*P0*V0 + (7.0/72.0)*Af*P0*Vf - 1.0/72.0*Af*P0*sqrt(-12*A0*P0 + 12*A0*Pf + 12*Af*P0 - 12*Af*Pf + 9*pow(V0, 2) + 18*V0*Vf + 9*pow(Vf, 2)) + (1.0/72.0)*Af*Pf*V0 - 7.0/72.0*Af*Pf*Vf + (1.0/72.0)*Af*Pf*sqrt(-12*A0*P0 + 12*A0*Pf + 12*Af*P0 - 12*Af*Pf + 9*pow(V0, 2) + 18*V0*Vf + 9*pow(Vf, 2)) - 1.0/12.0*pow(V0, 3) - 1.0/12.0*pow(V0, 2)*Vf + (1.0/36.0)*pow(V0, 2)*sqrt(-12*A0*P0 + 12*A0*Pf + 12*Af*P0 - 12*Af*Pf + 9*pow(V0, 2) + 18*V0*Vf + 9*pow(Vf, 2)) + (1.0/12.0)*V0*pow(Vf, 2) + (1.0/12.0)*pow(Vf, 3) - 1.0/36.0*pow(Vf, 2)*sqrt(-12*A0*P0 + 12*A0*Pf + 12*Af*P0 - 12*Af*Pf + 9*pow(V0, 2